# ДЗ4 Анна Головина группа БКЛ213

In [3]:
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
from nltk.corpus import stopwords
from nltk.tokenize import wordpunct_tokenize
import numpy as np
#from pymorphy2 import MorphAnalyzer
#from selenium import webdriver
#from sklearn.feature_extraction.text import TfidfVectorizer
#from sklearn.metrics.pairwise import cosine_similarity
import sqlite3
import re
import requests
import time
import tqdm

In [4]:
session = requests.session()
response = session.get('https://www.amalgama-lab.com')
response.status_code

200

Сначала я работала с сайтом https://tproger.ru/. Однако с сайта никак не удавалось достать нужное количество постов. Поскольку на сайте нет навигации по страницам, я искала по тегам, составив список со статьями на статьи данного раздела.

Проблема оказалась в том, что код я писала сама с минимальной опорой на конспекты (и слишком поздно осознала что можно было не мучиться и взять некоторые функции оттуда), а также не предусмотрела то, что одна статья может относиться к нескольким тегам сразу, то есть в базе получались дубликаты. Если дубликаты убрать, постов слишком мало (что видно из вывода ячейки). Я пыталась вручную добавлять в словарь те теги, которые, как мне казалось, не должны пересекаться, но все равно это очень костыльное решение, так что я в последний момент поменяла сайт, из-за чего люто пропустила дедлайн (тем более что доп информация у двух сайтов  не совпадала, и из-за этого, опять же просрочила дедлайн, за что каюсь, бью челом и слезно прошу прощения).

Тут, кстати, парсинг чисто в цикле, потому что пока я еще пыталась прописать функции, они в целом идейно соответствовали тому, что ниже реализовано для нового сайта (я просто ячейку восстанавливала постфактум из списка удаленных).

Итак, что же делает эта ячейка? Она ходит по словарю и собирает новости по разным тегам. Сначала создается пустой список, в который мы запишем потом ссылки на конкретные статьи.

Вторым циклом проходимся по ссылкам и вытаскиваем заголовок и текст. Еще вытаскивались всякие дополнительные штуки, но я потеряла ячейку, в которой было это все прописано.

In [89]:
from fake_useragent import UserAgent

dict_of_themes = {
    "Data Science": "data-science",
    "Дизайн интерфейсов и UX": "interface-design-ux",
    "Нейронные сети": "neural-network",
    "Математика и теория вероятностей": "math-and-probability",
    "Криптография": "crypto",
    "Искусственный интеллект": "ai",
    "DevOps": "devops",
    ".NET": "dot-net",
    "3D-принтер": "3d-printer",
    "Action Script": "action-script",
    "Amazon": "amazon",
    "Android": "android",
    "C#": "c-sharp",
    "C++": "cpp",
    "Cloudflare": "cloudflare",
    "CSS": "css",
    "Django": "django",
    "GitHub": "github",
    "GitLab": "gitlab",
    "Golang": "golang",
    "Jupyter Notebook": "jupyter-notebook",
}

articles = []

for theme in dict_of_themes.values():
    url = f"https://tproger.ru/tag/{theme}/"
    user_agent = UserAgent().chrome
    response = requests.get(url, headers={"User-Agent": user_agent})
    news_block = response.text
    soup = BeautifulSoup(news_block, "html.parser")

    hrefs = soup.find_all(class_="article__link")
    
    articles += [item.get("href") for item in hrefs]

for href in articles:
    req = requests.get(url=href, headers={"User-Agent": user_agent})
    page = req.text
    soup = BeautifulSoup(page, "html.parser")
    title = soup.find(class_="single__title").text
    text = soup.find(class_="single__content").text

print(len(articles))
#    print(f'СТАТЬЯ: {title}')
#    print(f'ТЕКСТ СТАТЬИ: {text}')

235


# Первое задание

*Выполнены требования по кол-ву постов*

Собрала данные с сайта https://elementy.ru/novosti_nauki, получилось 1108 постов (специально взяла с запасом).

автор, метки, количество комментариев

*Данные сохраняются в базу данных, есть описание, какие в ней поля и связи*

Структура базы данных:
в базе данных присутствует 5 таблиц, но много разнородных данных из них содержит только первая, остальное скорее для объема самой бд. Дальше пишу суть и название таблицы и после двоеточия колонки.

1. Тексты журнала (texts): id, title_id, title, comments_count, day, month, year, text
2. Автор (authors): author_id, author_name
3. Теги: tag_id, tag_name
4. Соединительная таблица (text_to_author): id_t_to_a, text_id, author_id
5. Еще одна соединительная таблица (text_to_ag): id_t_to_t, text_id, tag_id

Схему нарисовала на листочке, в репозиторий скинула.

*Посчитайте статистику:*

Взяла из задания (популярные теги), потому что у меня была идейно совпадающая таблица, так что я просто не видела смысла придумывать что-то еще.

Сначала посмотрим, что все в принципе хорошо и с сайтом можно установить соединение.

In [147]:
session = requests.session()
response = session.get('https://elementy.ru/novosti_nauki')
response.status_code

200

Сначала попробовала распарсить первую страницу, чтобы потом на такой код можно было навесить цикл и прогнать по нескольким страницам. Тут все подряд без функций, потому что на функции делила потом. А еще не надо обращать внимания на ошибку здесь в конце циклов, вот такие ошибки я потом на try-except отлавливала.

In [90]:
from fake_useragent import UserAgent

user_agent = UserAgent().chrome

url = "https://elementy.ru/novosti_nauki?page=1"
response = requests.get(url, headers={'User-Agent':user_agent})
news_block = response.text
soup = BeautifulSoup(news_block, "html.parser")

articles = []

hrefs = soup.find_all(class_="nohover")
articles += [f"https://elementy.ru{item.get('href')}" for item in hrefs]

for href in articles:
    req = requests.get(url=href, headers={"User-Agent":user_agent})
    page = req.text
    soup = BeautifulSoup(page, "html.parser")
    
    article_meta_info = soup.find(class_="mb itemhead newslist").text.split("•")
    article_title = article_meta_info[0].split("\n")[0]
    article_date = article_meta_info[0].split("\n")[1]
    article_author = article_meta_info[1]
    article_tags = article_meta_info[2].replace("\xa0", "").split(", ")
    article_comments = int(article_meta_info[3].split("\xa0")[0])
    article_text = soup.find(class_="memo").text
    
    print(
        f"ЗАГОЛОВОК: {article_title}\nДАТА: {article_date}\nАВТОР: {article_author}\nМЕТКИ: {article_tags}\nКОЛИЧЕСТВО КОММЕНТАРИЕВ: {article_comments}\nТЕКСТ СТАТЬИ: {article_text}\n"
    )

ЗАГОЛОВОК: Нобелевская премия по физике — 2022
ДАТА: 08.10.2022 
АВТОР:  Алексей Левин 
МЕТКИ: [' Нобелевские премии', 'Физика', 'Квантовая теория']
КОЛИЧЕСТВО КОММЕНТАРИЕВ: 64
ТЕКСТ СТАТЬИ: 



Лауреаты Нобелевской премии по физике 2022 года (слева направо): Ален Аспе (Alain Aspect), Джон Френсис Клаузер (John F. Clauser) и Антон Цайлингер (Anton Zeilinger). Все трое внесли важнейший склад в исследование явления квантовой спутанности и подтверждение того, что квантовый мир принципиально невозможно описать классическими методами. Фото с сайтов twitter.com, manilastandard.net и iqoqi-vienna.at

В этом году лауреатами Нобелевской премии по физике были объявлены француз Ален Аспе, американец Джон Клаузер и австриец Антон Цайлингер. В формулировке Нобелевского комитета указано, что эти ученые отмечены «за эксперименты со спутанными фотонами, которые продемонстрировали нарушение неравенств Белла и дали начало квантовой информатике». История этих исследований началась еще в середине 1930-х г

IndexError: list index out of range

Далее переходим к функциям.

Первая - краулер и парсер в одном флаконе. Принимает в качестве аргумента ссылку, возвращает html-разметку страницы по ссылке. Использовала как для обработки листа с блоком новостей, так и при парсинге статей (в этом случае это просто вложенная функция в page_parse).

In [91]:
# краулер-парсер
def crawl_parse(url):
    response = requests.get(url, headers={"User-Agent":user_agent})
    src = response.text
    crawl_soup = BeautifulSoup(src, "html.parser")
    return crawl_soup

Вторая - парсинг странички с новостями. Как аргумент принимает ссылку на страницу, возвращает словарь с заголовком статьи, разрезанной датой (и преобразованым месяцем), автором, тегами, количеством комментариев и текстом. Использовала исключительно при обработке статей.

In [92]:
# функция для парсинга конкретной страницы 
def page_parse(href):
    block = {}
    months = {
        "01":'янв',
        "02":'фев',
        "03":'мар',
        "04":'апр',
        "05":'мая',
        "06":'июн',
        "07":'июл',
        "08":'авг',
        "09":'сен',
        "10":'окт',
        "11":'ноя',
        "12":'дек'
    }
    page_soup = crawl_parse(href)
    block["id"] = href.split("/")[4]
    meta_info = page_soup.find(class_="mb itemhead newslist").text.split("•")
    block["title"] = title = meta_info[0].split("\n")[0]
    date = meta_info[0].split("\n")[1].replace("\xa0", "").split(".")
    block["day"] = date[0]
    block["month"] = months[date[1]]
    block["year"] = date[2]
    block["author"] = author = meta_info[1].replace("\xa0", "")
    block["tags"] = meta_info[2].replace("\xa0", "").split(", ")
    block["comments"] = int(meta_info[3].split("\xa0")[0])
    block["text"] = page_soup.find(class_="memo").text
    return block

Третья - одинаковые действия для ошибок разного рода. Как аргумент принимает ссылку статьи, в которой какая-то лажа и которая не пишется в базу данных, и название ошибки, которое вылетает в конструкции try-except, не возвращает ничего, но делает файл, куда пишутся странички, которые не скачались.

In [93]:
def exception(link, error):
    with open("error.txt", "a+") as file:
        data = f"{error}: {link}"
        file.write(data)
        file.write("\n")

Четвертая - запись в базу данных. Взяла из конспектов и адаптировала под свои таблицы и данные.

In [94]:
def write_to_db(block):
    tags = []

    cur.execute("SELECT tag_name, tag_id FROM tags")
    db_tags = {}
    for name, idx in cur.fetchall():
        db_tags[name] = idx

    for tag in block["tags"]:
        if tag not in db_tags:
            if db_tags.values():
                db_tags[tag] = max(db_tags.values()) + 1
            else:
                db_tags[tag] = 1
            cur.execute(
                "INSERT INTO tags (tag_id, tag_name) VALUES (?, ?)", (len(db_tags), tag)
            )
            conn.commit()
        tags.append(db_tags[tag])

    authors = []

    cur.execute("SELECT author_name, author_id FROM authors")
    db_authors = {}
    for name, idx in cur.fetchall():
        db_authors[name] = idx

    author = block["author"]
    if author not in db_authors:
        if db_authors.values():
            db_authors[author] = max(db_authors.values()) + 1
        else:
            db_authors[author] = 1
        cur.execute(
            "INSERT INTO authors (author_id, author_name) VALUES (?, ?)",
            (len(db_authors), author),
        )
        conn.commit()
    authors.append(db_authors[author])

    cur.execute(
        """
        INSERT INTO texts
            (title_id, title, comments_count, day, month, year, text) VALUES (?, ?, ?, ?, ?, ?, ?)
        """,
        (
            block["id"],
            block["title"],
            block["comments"],
            block["day"],
            block["month"],
            block["year"],
            block["text"]
        ),
    )
    conn.commit()

    cur.execute("SELECT id FROM texts WHERE title_id = ?", (block["id"],))
    text_id = cur.fetchone()[0]

    tags = [(text_id, t) for t in tags]

    cur.executemany("INSERT INTO text_to_tag (text_id, tag_id) VALUES (?, ?)", tags)

    authors = [(text_id, a) for a in authors]

    cur.executemany(
        "INSERT INTO text_to_author (text_id, author_id) VALUES (?, ?)", authors
    )

    conn.commit()

Создаем базу. ВАЖНО!!! База данных получилась больше 25МБ, так что гитхаб ее просто не пустил.

БАЗА ДАННЫХ ПО ССЫЛКЕ В ОБЛАКЕ: https://drive.google.com/drive/folders/1BFasuxFvFiq4Ink3HobU0SGHWD2hhaA0?usp=sharing

In [95]:
conn = sqlite3.connect("elementy.db")
cur = conn.cursor()

In [96]:
cur.execute(
    """
CREATE TABLE IF NOT EXISTS texts (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    title_id TEXT,
    title TEXT,
    comments_count INT,
    day INT,
    month INT,
    year INT,
    text TEXT
)
"""
)

cur.execute(
    """
CREATE TABLE IF NOT EXISTS authors (
    author_id INT,
    author_name TEXT
)
"""
)

cur.execute(
    """
CREATE TABLE IF NOT EXISTS tags (
    tag_id INT,
    tag_name INT
) 
"""
)

cur.execute(
    """
CREATE TABLE IF NOT EXISTS text_to_tag 
(id_t_to_t INTEGER PRIMARY KEY AUTOINCREMENT, text_id int, tag_id int)
"""
)

cur.execute(
    """
CREATE TABLE IF NOT EXISTS text_to_author
(id_t_to_a INTEGER PRIMARY KEY AUTOINCREMENT, text_id int, author_id int)
"""
)

conn.commit()

Основной код: здесь попытка замаскировать краулер и цикл, где и происходит сбор данных и запись в базу. На вход подается ссылка в виде форматированной строки для переключения между блоками с новостями. Далее реализуется функция *crawl_parse*, которая позволяет получить html блока с новостями. Далее я собираю статьи по классу nohover. Исходя из этого списка формирую еще один список со ссылками на статьи.

In [97]:
user_agent = UserAgent().chrome

articles = [] # список для записи ссылок на статьи и самих статей

for i in range(55):
    url_main = f"https://elementy.ru/novosti_nauki?page={i}"
    soup_main = crawl_parse(url_main)
    
    hrefs = soup_main.find_all(class_="nohover")

    articles+=[f"https://elementy.ru{item.get('href')}" for item in hrefs]

In [98]:
print(len(articles))

1650


Далее циклом прохожусь по полученному списку, применяя функцию page_parse через конструкцию try-except. При скачивании возникало несколько типовых ошибок:

1. list index out of range (это объясняется тем, что не ко всем статьям есть комментарии)
2. None has no attribute X (в случае если по параметрам, заданным в функции, где-нибудь что-то распарсилось как None, например, не указана дата или автор)

Все это сохраняется при записи в базу данных в отдельный файл, но в задании просили сдать только тетрадку и базу данных, так что не буду смущать лишними файлами.

In [100]:
for href in articles:
    try:
        data = page_parse(href)
        write_to_db(data)
    except Exception as e:
        exception(href, e)

Я не использовала tqdm, как в конспекте, потому что не вижу здесь в нем смысла, то есть код отработал и так достаточно быстро для объема получившейся базы.

Наконец, статистика (самые популярные теги, 30 штук).

In [109]:
tag_query = """
SELECT count(text_to_tag.tag_id) as t, tags.tag_name 
    FROM text_to_tag 
        JOIN tags ON tags.tag_id = text_to_tag.tag_id 
            GROUP BY text_to_tag.tag_id 
            ORDER BY t DESC
            LIMIT 30;
"""
res = cur.execute(tag_query).fetchall()
for i in res:
    print(f"Тег: {i[1]}; количество употреблений: {i[0]}")

Тег: Эволюция; количество употреблений: 192
Тег: Генетика; количество употреблений: 131
Тег:  Палеонтология; количество употреблений: 129
Тег:  Генетика; количество употреблений: 93
Тег:  Эволюция; количество употреблений: 91
Тег:  Орнитология; количество употреблений: 80
Тег: Зоология; количество употреблений: 67
Тег: Наука в России; количество употреблений: 57
Тег:  Этология; количество употреблений: 52
Тег:  Геология; количество употреблений: 52
Тег: Микробиология; количество употреблений: 50
Тег: Энтомология; количество употреблений: 49
Тег:  Нейробиология; количество употреблений: 42
Тег:  Химия; количество употреблений: 41
Тег: Палеонтология; количество употреблений: 41
Тег: Медицина; количество употреблений: 39
Тег: Геология; количество употреблений: 38
Тег:  Антропология; количество употреблений: 37
Тег:  Астрономия; количество употреблений: 36
Тег: Этология; количество употреблений: 35
Тег: Физика; количество употреблений: 32
Тег: Орнитология; количество употреблений: 30
Тег: 

# Второе задание

*Код программы разделен на логичные функции (а не все подряд в цикле)*

Ну вроде бы да...

*Есть обработка ошибок (try-except), куда-то сохраняется информация о том, какие посты не скачались и какая ошибка*

Обработка есть, правда только в одном месте, но в моем коде эту конструкцию больше особо никуда не применишь. Те статьи, которые не скачались, сохраняются в txt-файл в виде *ошибка: текст*.

*С помощью TF-IDF выделите ключевые слова для каждой новости*

Ключевые слова (по 10) нашла по коду из конспектов и сначала я даже хотела сохранить в новую таблицу базы данных на всякий случай в виде новой таблички. Но база и так тяжелая (28МБ как-никак, гитхаб в шоке), так что просто вывела на печать в соответствии с номером, который является id в таблице *texts* в базе данных.

Код здесь не поясняю, т.к. это просто дз2, в котором переменная articles_texts содержит тексты статей, которые я достала запросом.

In [103]:
morph = MorphAnalyzer()
stop_words = set(stopwords.words("russian"))

In [104]:
def get_top_tf_idf_words(tfidf_vector, feature_names, top_n):
    sorted_nzs = np.argsort(tfidf_vector.data)[:-(top_n+1):-1]
    return feature_names[tfidf_vector.indices[sorted_nzs]]

In [105]:
text_query = """
SELECT text
FROM texts
"""

articles_texts = [text[0] for text in cur.execute(text_query).fetchall()]

articles_preprocessed = []

for a_text in articles_texts:
    a_tokens = wordpunct_tokenize(a_text)
    a_lemmatized = " ".join([morph.parse(item)[0].normal_form for item in a_tokens])
    articles_preprocessed.append(a_lemmatized)

In [106]:
vectorizer = TfidfVectorizer(
    analyzer="word", stop_words=stop_words
)

In [107]:
tfidf = vectorizer.fit_transform(articles_preprocessed)

In [108]:
feature_names = np.array(vectorizer.get_feature_names())

for i, article in enumerate(articles_texts):
    vector = tfidf[i, :]
    words = get_top_tf_idf_words(vector, feature_names, 10)
    print(f"{i+1}: {words}")

1: ['куколка' 'муравей' 'личинка' 'жидкость' 'biroi' 'линочной' 'когорта'
 'взрослый' 'рейдер' 'колония']
2: ['чума' 'ген' 'смерть' 'иммунный' 'цыган' 'лимфоцит' 'чёрный' 'лондон'
 'захоронение' 'останки']
3: ['подёнка' 'личинка' 'hexagenia' 'насекомое' 'sharephemeridae' 'водный'
 'массовый' 'саранча' 'норка' 'миссисипи']
4: ['уэбба' 'шокли' 'транзистор' 'браттейн' 'год' 'наса' 'бардин' 'усилитель'
 'джеймс' 'космический']
5: ['змея' 'живорождение' 'месселофис' 'удав' 'детёныш' 'эмбрион' 'рептилия'
 'скелет' 'viviparity' 'скорлупа']
6: ['афелида' 'гриб' 'циста' 'зооспора' 'клетка' 'розелла' 'водоросль'
 'фагоцитоз' 'паразит' 'wash']
7: ['песня' 'мухоловка' 'пеструшка' 'диалект' 'белошеек' 'птенец' 'пение'
 'популяция' 'слышать' 'вид']
8: ['ледник' 'подлёдный' 'субгляциальный' 'антарктида' 'лёд' 'таяние' 'поток'
 'вода' 'море' 'основание']
9: ['самоконтроль' 'сойка' 'gratification' 'delay' 'птица' 'терпеливый'
 'способность' 'когнитивный' 'лакомство' 'тест']
10: ['шмель' 'игровой' 'шари

# Третье задание

*Используя sklearn.metrics.pairwise.cosine_similarity, посчитайте близость между tf-idf векторами текстов и для каждого текста найдите 3 похожих. Сохраните пары в БД, в тетрадке приведите результаты для 3 текстов (можно вывести только начало текста).*

Я честно пыталась понять как организовать код, но здесь оставляю скорее попытку решить задание. То есть, я так понимаю, можно прогнать через цикл все статьи и составить словарь с ключами-векторами и значениями-текстами. Но вот на этом моменте у меня сломался мозг, я поняла, что не помню, как это привести к нормальному виду и сдалась.

In [72]:
vect_to_text = {}

for text in articles_texts:
    item = []
    item.append(text)
    vect = vectorizer.fit_transform(item)
    vect_to_text[vect] = text

TypeError: unhashable type: 'csr_matrix'

То есть я понимаю, что можно задать списком вектора для сравнения. Например, если есть два вектора a и b, то если жмакнуть на эту ячейку, вылетит красивая матрица.

In [ ]:
cos_sim = cosine_similarity([a, b])
print(cos_sim)

Но я уже не способна мыслить. А так, конечно, можно было бы составлять матрицы для каждой статьи и искать самое близкое значение.

# Четвертое задание

*Посмотрите, как запустить Selenium в конспекте. Возьмите любой сайт и напишите код, который будет скроллить его страницу.*

Для начала стоит сказать, что это безобразие выполняет не совсем питон, а JavaScript.

Организовала бесконечный скролл, а не с конечным числом итераций. Сначала устанавливаем веб-драйвер, присваивая значение переменной browser, затем даем ссылку на сайт через метод .get() (я выбрала злосчастный *tproger.ru*).

Далее необходимо посчитать высоту исходной странички и сохранить в переменную limit. Для этого использую метод .scrollHeight, который измеряет высоту заданного элемента, а задаем мы document.body (это элемент, соответствующий содержимому тега <body> или <frameset> в html-разметке; в нашем случае <body>). Вся эта прелесть зашивается в метод .execute_script(), который буквально связывает описанные команды и веб-драйвер и передает драйверу данные, полученные в результате работы метода .scrollHeight относительно элемента document.body. После этого запускаем бесконечный цикл:

1. Страница скроллится до низа (то есть по уже высчитанной высоте, соответственно, далее подгружаются новые посты, и высота страницы меняется). Использую все тот же метод .execute_script(), чтобы передать данные драйверу. В него зашиваю команду .scrollTo(), то есть буквально "листать вниз"; в качестве параметров указываю начало, откуда скроллить (нулевая точка полученного ну так скажем отрезка длины страницы), и конец (document.body.scrollHeight, который, как мы уже поняли, возвращает длину страницы, то есть здесь можно получить конечную точку, число, соответствующее длине страницы). Команда .scrollTo() выполняется теперь не относительно элемента в html-разметке, а относительно всего окна, так что window.scrollTo(). Даю передышку и время подгрузить новые посты (5 секунд).

2. Считаем новую высоту странички теми же методами и присваиваем переменной new_limit.

3. Если постов больше нет (т.е. если новый лимит совпадает со старым), то все, прекращаем работу, если нет, то работаем дальше.

4. Переприсваиваем значение новой высоты переменной limit.

На этом все. Я не знаю, что еще можно сказать про работу этого блока.

Не использовала tqdm, потому что у меня скролл бесконечный, и заранее неизвестны границы, до которых дойдет эта шайтан-машина.

Могу разве что сказать про результат работы. Страница скроллится достаточно резко и рвано, но в целом это не мешает работе кода и, например, меня не смущает.

In [111]:
browser = webdriver.Chrome()
browser.get("https://tproger.ru/")
limit = browser.execute_script("return document.body.scrollHeight")
while True:
    browser.execute_script("window.scrollTo(0,document.body.scrollHeight)")
    time.sleep(5)
    new_limit = browser.execute_script("return document.body.scrollHeight")
    if new_limit == limit:
        break
    limit = new_limit

Код отработал без ошибок. Я не очень понимаю, как добавить в тетрадку результат работы этого куска, так что оставлю просто код.